# 🤖 AI Incident Database — Master Dataset Builder
### Reproducible pipeline: CSET + GMF + MIT + Incidents → cleaned, joined master

---

## 👋 How to use this notebook

**For most users — just run everything:**
1. Open this notebook in [Google Colab](https://colab.research.google.com)
2. Click **Runtime → Run all** (or press `Ctrl+F9`)
3. Wait ~2 minutes — the master dataset will auto-download to your computer

**If a source file's column has been renamed or a new column was added:**
- Run **Cell 1 (⚙️ Configuration)** and **Cell 3 (📥 Download)** first
- Then run **Cell 4 (🔍 Schema Health Check)** — it will show you exactly which column names have changed
- Update the relevant mapping in **Cell 1** and re-run from Cell 4 onwards

---
| Cell | What it does | Edit needed? |
|------|-------------|-------------|
| **1 · ⚙️ Configuration** | All settings & column mappings in one place | ✏️ Only cell you ever edit |
| **2 · 📦 Install & Import** | Installs libraries, nothing to change | ❌ Never |
| **3 · 📥 Download Snapshot** | Downloads latest AIID data automatically | ❌ Never |
| **4 · 🔍 Schema Health Check** | Validates column names — flags any renames | ❌ Never |
| **5 · 📂 Load Raw Data** | Reads all CSVs into memory | ❌ Never |
| **6–9 · 🧹 Clean each source** | Standardises, drops noise columns | ❌ Never |
| **10 · 🔗 Build Master** | Joins all sources into one table | ❌ Never |
| **11 · ✅ Validation** | Automated quality checks | ❌ Never |
| **12 · 💾 Export** | Saves formatted Excel + triggers download | ❌ Never |
| **13 · 📊 Summary** | Final statistics | ❌ Never |

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  ⚙️  CELL 1 — CONFIGURATION                                                 ║
# ║  This is the ONLY cell you ever need to edit.                               ║
# ║  Everything below is automatically used by the rest of the notebook.        ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

from pathlib import Path

# ── 1A. FILE PATHS ─────────────────────────────────────────────────────────────
# Where to save the downloaded snapshot and final output.
# In Google Colab, files are saved under /content/ by default.

SNAPSHOT_DIR = Path("/content/aiid_snapshot")   # ← where downloaded files live
OUTPUT_PATH  = Path("/content/AIID_Master_Dataset.xlsx")  # ← final Excel file

# ── 1B. DOWNLOAD SETTINGS ──────────────────────────────────────────────────────
# Leave these as-is unless the AIID website structure changes.

BASE_URL          = "https://incidentdatabase.ai"
SNAPSHOT_PAGE_URL = f"{BASE_URL}/research/snapshots/"
SNAPSHOT_FILTER   = ".tar.bz2"

# ── 1C. COLUMN MAPPINGS ────────────────────────────────────────────────────────
# Format:  "Original column name in source file" : "Name to use in master dataset"
#
# HOW TO UPDATE: If the Schema Health Check (Cell 4) flags a column as missing,
# find that column in the dict below and update the KEY (left side) to match
# the new column name shown in the health check output.
# The VALUE (right side = master column name) should stay the same.

# --- incidents.csv ---
INCIDENTS_COLUMNS = {
    # original name in file          : master column name
    "incident_id"                    : "Incident ID",
    "date"                           : "date",
    "title"                          : "title",
    "description"                    : "description",
    "reports"                        : "reports",        # used to count reports, then dropped
    "Alleged deployer of AI system"  : "deployer_raw",   # parsed to readable text
    "Alleged developer of AI system" : "developer_raw",
    "Alleged harmed or nearly harmed parties" : "harmed_raw",
}

# --- classifications_MIT.csv ---
MIT_COLUMNS = {
    "Incident ID"    : "Incident ID",
    "Risk Domain"    : "Risk Domain",
    "Risk Subdomain" : "Risk Subdomain",
    "Entity"         : "Responsible Entity",
    "Timing"         : "Timing",
    "Intent"         : "Intent",
}

# --- classifications_GMF.csv ---
GMF_COLUMNS = {
    "Incident ID"                : "Incident ID",
    "Known AI Goal"              : "AI Goal",
    "Known AI Technology"        : "AI Technology",
    "Known AI Technical Failure" : "Technical Failure",
}

# --- classifications_CSETv1.csv ---
CSET_COLUMNS = {
    "Incident ID"                      : "Incident ID",
    "Harm Domain"                      : "Harm Domain",
    "Tangible Harm"                    : "Tangible Harm",
    "AI Harm Level"                    : "AI Harm Level",
    "Rights Violation"                 : "Rights Violation",
    "Harm Distribution Basis"          : "Harm Distribution Basis",
    "Special Interest Intangible Harm" : "Special Interest Intangible Harm",
    "Lives Lost"                       : "Lives Lost",
    "Injuries"                         : "Injuries",
    "Estimated Harm Quantities"        : "Estimated Harm Quantities",
    "Sector of Deployment"             : "Sector of Deployment",
    "Public Sector Deployment"         : "Public Sector Deployment",
    "Infrastructure Sectors"           : "Infrastructure Sectors",
    "Location Region"                  : "Location Region",
    "Location Country (two letters)"   : "Country Code",
    "Location City"                    : "Location City",
    "Date of Incident Year"            : "Incident Year",
    "Date of Incident Month"           : "Incident Month",
    "Date of Incident Day"             : "Incident Day",
    "Intentional Harm"                 : "Intentional Harm",
    "Autonomy Level"                   : "Autonomy Level",
    "AI Task"                          : "AI Task",
    "AI tools and methods"             : "AI Methods",
    "AI System Description"            : "AI System Description",
    "Data Inputs"                      : "Data Inputs",
    "Physical Objects"                 : "Physical Objects",
    "Deployed"                         : "Deployed",
    "Involving Minor"                  : "Involving Minor",
    "Protected Characteristic"         : "Protected Characteristic",
    "AI System"                        : "AI System",
    "Detrimental Content"              : "Detrimental Content",
    "Impact on Critical Services"      : "Impact on Critical Services",
    "Multiple AI Interaction"          : "Multiple AI Interaction",
    "Embedded"                         : "Embedded",
    "Entities"                         : "Entities",
    "Estimated Date"                   : "Estimated Date",
}

# --- duplicates.csv ---
DUPLICATES_ID_COLUMN = "duplicate_incident_number"   # column holding IDs to remove

# ── 1D. VALIDATION THRESHOLDS ─────────────────────────────────────────────────
# Adjust these if the database grows significantly.

EXPECTED_MIN_INCIDENTS = 1300    # minimum expected row count after deduplication
EXPECTED_MIT_COVERAGE  = 85.0    # minimum % of incidents with MIT Risk Domain

# ── 1E. MASTER COLUMN ORDER ────────────────────────────────────────────────────
# Controls column order in the output Excel file.
# Columns not listed here are appended at the end automatically.

MASTER_COLUMN_ORDER = [
    # Core identity (100% filled — always use these for analysis)
    "Incident ID", "date", "year", "title", "description",
    "deployer", "developer", "harmed",
    # Coverage flag
    "Data Sources", "report_count",
    # MIT taxonomy (91% coverage)
    "Risk Domain", "Risk Subdomain", "Responsible Entity", "Intent", "Timing",
    # GMF taxonomy (24% coverage)
    "AI Goal", "AI Technology", "Technical Failure",
    # CSET taxonomy (16% coverage)
    "Harm Domain", "Tangible Harm", "AI Harm Level",
    "Rights Violation", "Harm Distribution Basis",
    "Lives Lost", "Injuries",
    "Sector of Deployment", "Public Sector Deployment", "Infrastructure Sectors",
    "Location Region", "Country Code", "Location City",
    "Incident Year", "Incident Month", "Incident Day",
    "Intentional Harm", "Autonomy Level",
    "AI Task", "AI Methods", "AI System Description", "Data Inputs",
    "Involving Minor", "Protected Characteristic",
    "AI System", "Detrimental Content", "Impact on Critical Services",
    "Multiple AI Interaction", "Embedded", "Entities",
    "Physical Objects", "Deployed", "Estimated Date",
    "Estimated Harm Quantities", "Special Interest Intangible Harm",
]

print("✅  Configuration loaded")
print(f"    Snapshot dir  : {SNAPSHOT_DIR}")
print(f"    Output file   : {OUTPUT_PATH}")
print(f"    Column mappings: incidents={len(INCIDENTS_COLUMNS)}  MIT={len(MIT_COLUMNS)}  "
      f"GMF={len(GMF_COLUMNS)}  CSET={len(CSET_COLUMNS)}")

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  📦  CELL 2 — INSTALL & IMPORT  (no editing needed)                         ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

import subprocess, sys

for pkg in ["beautifulsoup4", "lxml", "requests", "openpyxl"]:
    subprocess.run([sys.executable, "-m", "pip", "install", pkg, "-q"], check=True)

import ast, re, tarfile, collections
import requests
import pandas as pd
from bs4 import BeautifulSoup
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter

# Google Colab download helper (no-op outside Colab)
try:
    from google.colab import files as colab_files
    IN_COLAB = True
except ImportError:
    colab_files = None
    IN_COLAB = False

print(f"✅  Libraries ready  |  Running in {'Google Colab' if IN_COLAB else 'local Jupyter'}")

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  📥  CELL 3 — DOWNLOAD LATEST SNAPSHOT  (no editing needed)                 ║
# ║  Automatically finds and downloads the most recent AIID data export.        ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

SNAPSHOT_DIR.mkdir(parents=True, exist_ok=True)

# 1. Scrape the snapshots page for all .tar.bz2 links
print(f"Fetching snapshot list from {SNAPSHOT_PAGE_URL} ...")
html  = requests.get(SNAPSHOT_PAGE_URL, timeout=60).text
soup  = BeautifulSoup(html, "lxml")
links = []
for a in soup.select("a[href]"):
    href = a["href"]
    if SNAPSHOT_FILTER in href:
        if href.startswith("/"):
            href = BASE_URL + href
        links.append(href)

if not links:
    raise RuntimeError(
        "No snapshot links found on the page — the site structure may have changed. "
        f"Check {SNAPSHOT_PAGE_URL} manually."
    )

# 2. Pick the latest by timestamp (backup-YYYYMMDDHHMMSS.tar.bz2)
def _ts(url):
    m = re.search(r"backup-(\d{14})\.tar\.bz2", url)
    return m.group(1) if m else ""

latest_url = sorted(links, key=_ts)[-1]
print(f"Latest snapshot  : {latest_url}")

# 3. Download with progress indication
tar_path = SNAPSHOT_DIR / latest_url.split("/")[-1]
if tar_path.exists():
    print(f"Already downloaded: {tar_path.name} — skipping download")
else:
    print("Downloading ...", end=" ", flush=True)
    with requests.get(latest_url, stream=True, timeout=300) as r:
        r.raise_for_status()
        total  = int(r.headers.get("content-length", 0))
        done   = 0
        with open(tar_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=1024 * 512):
                if chunk:
                    f.write(chunk)
                    done += len(chunk)
                    if total:
                        print(f"\r  {done/1024/1024:.1f} MB / {total/1024/1024:.1f} MB",
                              end="", flush=True)
    print(f"\n✅  Downloaded: {tar_path.name}")

# 4. Extract
print("Extracting ...", end=" ", flush=True)
with tarfile.open(tar_path, "r:bz2") as tar:
    tar.extractall(SNAPSHOT_DIR, filter="data")  # 'data' filter suppresses Python 3.14 warning
print("done")

# 5. Locate the five CSVs and store paths
def _find(pattern):
    matches = list(SNAPSHOT_DIR.rglob(pattern))
    if not matches:
        raise FileNotFoundError(
            f"Could not find '{pattern}' under {SNAPSHOT_DIR}. "
            "The snapshot may have a different internal structure."
        )
    return matches[0]

INCIDENTS_PATH  = _find("incidents.csv")
MIT_PATH        = _find("classifications_MIT.csv")
GMF_PATH        = _find("classifications_GMF.csv")
CSET_PATH       = _find("classifications_CSETv1.csv")
DUPLICATES_PATH = _find("duplicates.csv")

print("\nLocated files:")
for label, p in [("incidents", INCIDENTS_PATH), ("MIT", MIT_PATH),
                  ("GMF", GMF_PATH), ("CSET", CSET_PATH), ("duplicates", DUPLICATES_PATH)]:
    print(f"  {label:<12} → {p.name}")

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  🔍  CELL 4 — SCHEMA HEALTH CHECK  (no editing needed)                      ║
# ║                                                                              ║
# ║  Compares every column in your Cell 1 mappings against the actual files.    ║
# ║  If a source file has been updated with renamed columns, this cell shows    ║
# ║  you exactly which ones changed so you can fix Cell 1 and re-run.           ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

# Load just the headers of each file (fast — no data read)
_raw_incidents  = pd.read_csv(INCIDENTS_PATH,  nrows=0)
_raw_mit        = pd.read_csv(MIT_PATH,        nrows=0)
_raw_gmf        = pd.read_csv(GMF_PATH,        nrows=0)
_raw_cset       = pd.read_csv(CSET_PATH,       nrows=0)
_raw_duplicates = pd.read_csv(DUPLICATES_PATH, nrows=0)

file_cols = {
    "incidents"  : set(_raw_incidents.columns),
    "MIT"        : set(_raw_mit.columns),
    "GMF"        : set(_raw_gmf.columns),
    "CSETv1"     : set(_raw_cset.columns),
    "duplicates" : set(_raw_duplicates.columns),
}

check_maps = {
    "incidents" : INCIDENTS_COLUMNS,
    "MIT"       : MIT_COLUMNS,
    "GMF"       : GMF_COLUMNS,
    "CSETv1"    : CSET_COLUMNS,
    "duplicates": {DUPLICATES_ID_COLUMN: DUPLICATES_ID_COLUMN},
}

all_ok    = True
issues    = []
new_cols  = []

SEP = "─" * 68
print(SEP)
print("  🔍  SCHEMA HEALTH CHECK")
print(SEP)

for source, mapping in check_maps.items():
    actual_cols   = file_cols[source]
    expected_keys = set(mapping.keys())
    missing       = expected_keys - actual_cols      # in config but not in file
    added         = actual_cols   - expected_keys    # in file but not in config

    # Filter noise from added (admin/notes columns we intentionally skip)
    _noise = {"Namespace", "Published", "Incident Number", "_id",
               "Annotator", "Peer Reviewer", "Annotation Status", "Quality Control"}
    added_meaningful = added - _noise - {c for c in added if
                                          "Snippet" in c or "Discussion" in c
                                          or "Notes" in c or "Potential" in c}

    status = "✅" if not missing else "❌"
    print(f"\n  {status}  {source}  ({len(actual_cols)} columns in file)")

    if missing:
        all_ok = False
        for col in sorted(missing):
            issues.append((source, col, mapping[col]))
            # Suggest close matches
            close = [c for c in actual_cols
                     if col.lower().replace(" ", "") in c.lower().replace(" ", "")
                     or c.lower().replace(" ", "") in col.lower().replace(" ", "")]
            hint  = f"  → Maybe renamed to: {close}" if close else ""
            print(f"      ❌  MISSING: '{col}' (maps to master '{mapping[col]}'){hint}")

    if added_meaningful:
        for col in sorted(added_meaningful):
            new_cols.append((source, col))
            print(f"      ℹ️   NEW COLUMN not yet mapped: '{col}'")

print(f"\n{SEP}")
if all_ok:
    print("  ✅  All column mappings match — safe to continue")
else:
    print(f"  ❌  {len(issues)} column(s) not found in source files")
    print("")
    print("  HOW TO FIX:")
    print("  1. Look at the MISSING entries above and note the new column name")
    print("  2. Go back to Cell 1 (⚙️ Configuration)")
    print("  3. Find the old name in the relevant mapping dict and update it")
    print("  4. Re-run from Cell 1 onwards")
if new_cols:
    print(f"\n  ℹ️   {len(new_cols)} new column(s) exist in source files but aren't mapped.")
    print("  To include them, add entries to the relevant dict in Cell 1.")
print(SEP)

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  📂  CELL 5 — LOAD RAW DATA  (no editing needed)                            ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

incidents_raw  = pd.read_csv(INCIDENTS_PATH,  low_memory=False)
mit_raw        = pd.read_csv(MIT_PATH,        low_memory=False)
gmf_raw        = pd.read_csv(GMF_PATH,        low_memory=False)
cset_raw       = pd.read_csv(CSET_PATH,       low_memory=False)
duplicates_raw = pd.read_csv(DUPLICATES_PATH)

print("Raw file dimensions:")
print(f"  incidents.csv           {incidents_raw.shape[0]:>5} rows  {incidents_raw.shape[1]:>3} cols")
print(f"  classifications_MIT     {mit_raw.shape[0]:>5} rows  {mit_raw.shape[1]:>3} cols")
print(f"  classifications_GMF     {gmf_raw.shape[0]:>5} rows  {gmf_raw.shape[1]:>3} cols")
print(f"  classifications_CSETv1  {cset_raw.shape[0]:>5} rows  {cset_raw.shape[1]:>3} cols")
print(f"  duplicates              {duplicates_raw.shape[0]:>5} rows")

# IDs to remove (duplicates list)
dup_ids = set(duplicates_raw[DUPLICATES_ID_COLUMN].dropna().astype(int).tolist())
print(f"\n  Duplicate IDs to remove : {len(dup_ids)}")

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  🧹  CELL 6 — CLEAN: incidents.csv  (no editing needed)                     ║
# ║                                                                              ║
# ║  What this does:                                                             ║
# ║  • Removes known duplicate incidents                                         ║
# ║  • Drops MongoDB internal _id field                                          ║
# ║  • Parses date → datetime and extracts year                                  ║
# ║  • Counts linked news reports per incident (report_count)                   ║
# ║  • Converts JSON slug lists for deployer/developer/harmed to readable text  ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

def _parse_entity_list(val):
    """Convert JSON slug list e.g. ['youtube'] → 'Youtube'"""
    try:
        items = ast.literal_eval(str(val))
        return ", ".join(str(i).replace("-", " ").strip().title() for i in items)
    except Exception:
        return str(val).strip()

def _count_reports(val):
    try:
        return len(ast.literal_eval(str(val)))
    except Exception:
        return 0

# Select only mapped columns (drop _id and anything unmapped)
keep_inc = [c for c in INCIDENTS_COLUMNS if c in incidents_raw.columns]
inc = incidents_raw[keep_inc].copy()
inc = inc.rename(columns=INCIDENTS_COLUMNS)

# Remove duplicates
before = len(inc)
inc = inc[~inc["Incident ID"].isin(dup_ids)].copy()
print(f"Incidents after removing duplicates: {before} → {len(inc)}")

# Parse date & year
inc["date"] = pd.to_datetime(inc["date"], errors="coerce")
inc["year"] = inc["date"].dt.year.astype("Int64")

# Count reports
inc["report_count"] = inc["reports"].apply(_count_reports)
print(f"Report count — min: {inc['report_count'].min()}  "
      f"max: {inc['report_count'].max()}  mean: {inc['report_count'].mean():.1f}")

# Parse entity slugs → readable
inc["deployer"]  = inc["deployer_raw"].apply(_parse_entity_list)
inc["developer"] = inc["developer_raw"].apply(_parse_entity_list)
inc["harmed"]    = inc["harmed_raw"].apply(_parse_entity_list)

# Drop raw working columns
inc = inc.drop(columns=["reports", "deployer_raw", "developer_raw", "harmed_raw"],
               errors="ignore")
inc = inc.sort_values("Incident ID").reset_index(drop=True)

print(f"\nCleaned incidents: {inc.shape[0]} rows × {inc.shape[1]} cols")
print(f"Columns: {inc.columns.tolist()}")

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  🧹  CELL 7 — CLEAN: classifications_MIT.csv  (no editing needed)           ║
# ║                                                                              ║
# ║  What this does:                                                             ║
# ║  • Keeps only columns defined in MIT_COLUMNS (Cell 1)                       ║
# ║  • Strips numeric prefixes from domain labels                               ║
# ║    e.g.  "1. Discrimination and Toxicity" → "Discrimination and Toxicity"  ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

keep_mit = [c for c in MIT_COLUMNS if c in mit_raw.columns]
mit_c = mit_raw[keep_mit].copy()
mit_c = mit_c.rename(columns=MIT_COLUMNS)
mit_c = mit_c[~mit_c["Incident ID"].isin(dup_ids)]

# Strip numeric prefixes for readability
if "Risk Domain" in mit_c.columns:
    mit_c["Risk Domain"]    = mit_c["Risk Domain"].str.replace(r"^\d+\.\s*", "", regex=True)
if "Risk Subdomain" in mit_c.columns:
    mit_c["Risk Subdomain"] = mit_c["Risk Subdomain"].str.replace(r"^\d+\.\d+\.\s*", "", regex=True)

mit_c = mit_c.sort_values("Incident ID").reset_index(drop=True)

print(f"Cleaned MIT: {mit_c.shape[0]} rows × {mit_c.shape[1]} cols")
print(f"Risk Domains ({mit_c['Risk Domain'].nunique()} unique):")
for domain, count in mit_c["Risk Domain"].value_counts().items():
    print(f"  {count:4d}  {domain}")

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  🧹  CELL 8 — CLEAN: classifications_GMF.csv  (no editing needed)           ║
# ║                                                                              ║
# ║  What this does:                                                             ║
# ║  • Keeps only the 3 confirmed classification columns (from GMF_COLUMNS)     ║
# ║  • Drops Snippet, Discussion, and Potential columns (annotator working      ║
# ║    material — not structured data for analysis)                             ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

keep_gmf = [c for c in GMF_COLUMNS if c in gmf_raw.columns]
gmf_c = gmf_raw[keep_gmf].copy()
gmf_c = gmf_c.rename(columns=GMF_COLUMNS)
gmf_c = gmf_c[~gmf_c["Incident ID"].isin(dup_ids)]
gmf_c = gmf_c.sort_values("Incident ID").reset_index(drop=True)

print(f"Cleaned GMF: {gmf_c.shape[0]} rows × {gmf_c.shape[1]} cols")
print(f"Columns: {gmf_c.columns.tolist()}")

print(f"\nTop 10 Technical Failures:")
failures = []
for v in gmf_c["Technical Failure"].dropna():
    failures.extend(f.strip() for f in str(v).split(","))
for fail, cnt in collections.Counter(failures).most_common(10):
    print(f"  {cnt:3d}  {fail}")

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  🧹  CELL 9 — CLEAN: classifications_CSETv1.csv  (no editing needed)        ║
# ║                                                                              ║
# ║  What this does:                                                             ║
# ║  • Keeps only columns defined in CSET_COLUMNS (Cell 1)                      ║
# ║  • Drops admin, notes, and annotator-workflow columns automatically         ║
# ║  • Normalises Location City blanks to null                                  ║
# ║  • Normalises Incident Month from mixed formats ("3", " 3") → "March"       ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

keep_cset = [c for c in CSET_COLUMNS if c in cset_raw.columns]
cset_c = cset_raw[keep_cset].copy()
cset_c = cset_c.rename(columns=CSET_COLUMNS)
cset_c = cset_c[~cset_c["Incident ID"].isin(dup_ids)]
cset_c = cset_c.drop_duplicates(subset="Incident ID")

# Normalise Location City
if "Location City" in cset_c.columns:
    cset_c["Location City"] = cset_c["Location City"].str.strip().replace("", pd.NA)

# Normalise Incident Month: numeric strings → full month names
if "Incident Month" in cset_c.columns:
    month_names = ["January","February","March","April","May","June",
                   "July","August","September","October","November","December"]
    month_map   = {str(i): name for i, name in enumerate(month_names, 1)}
    month_map.update({f" {k}": v for k, v in month_map.items()})  # handle leading space
    cset_c["Incident Month"] = cset_c["Incident Month"].replace(month_map)

cset_c = cset_c.sort_values("Incident ID").reset_index(drop=True)

print(f"Cleaned CSET: {cset_c.shape[0]} rows × {cset_c.shape[1]} cols")
print(f"Columns: {cset_c.columns.tolist()}")

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  🔗  CELL 10 — BUILD MASTER DATASET  (no editing needed)                    ║
# ║                                                                              ║
# ║  Join strategy: incidents is the spine. All taxonomy tables join via        ║
# ║  LEFT JOIN on Incident ID — every incident is preserved even if not         ║
# ║  classified by all sources.                                                 ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

# 1. Core identity columns
inc_core = inc[["Incident ID","date","year","title","description",
                "deployer","developer","harmed","report_count"]].copy()

# 2. Data Sources provenance flag
mit_ids  = set(mit_c["Incident ID"])
gmf_ids  = set(gmf_c["Incident ID"])
cset_ids = set(cset_c["Incident ID"])

def _sources(iid):
    parts = []
    if iid in mit_ids:  parts.append("MIT")
    if iid in gmf_ids:  parts.append("GMF")
    if iid in cset_ids: parts.append("CSETv1")
    return " | ".join(parts) if parts else "None"

inc_core["Data Sources"] = inc_core["Incident ID"].apply(_sources)

# 3. Join MIT
mit_join = mit_c[["Incident ID","Risk Domain","Risk Subdomain",
                   "Responsible Entity","Intent","Timing"]]
master = inc_core.merge(mit_join, on="Incident ID", how="left")

# 4. Join GMF
gmf_join = gmf_c[["Incident ID","AI Goal","AI Technology","Technical Failure"]]
master = master.merge(gmf_join, on="Incident ID", how="left")

# 5. Join CSET (all cleaned columns)
cset_join = ["Incident ID"] + [c for c in cset_c.columns if c != "Incident ID"]
master = master.merge(cset_c[cset_join], on="Incident ID", how="left")

master = master.sort_values("Incident ID").reset_index(drop=True)

# 6. Enforce column order from config (extra columns appended at end)
ordered = [c for c in MASTER_COLUMN_ORDER if c in master.columns]
extra   = [c for c in master.columns if c not in ordered]
if extra:
    print(f"ℹ️   Extra columns (appended at end): {extra}")
master = master[ordered + extra]

print(f"✅  Master dataset: {len(master):,} rows × {len(master.columns)} columns")
print(f"\nCoverage breakdown:")
for src, cnt in master["Data Sources"].value_counts().items():
    pct = cnt / len(master) * 100
    bar = "█" * int(pct / 4)
    print(f"  {src:<28} {cnt:>5,}  ({pct:.1f}%)  {bar}")

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  ✅  CELL 11 — VALIDATION  (no editing needed)                               ║
# ║  Automated checks before export. Fix any ❌ errors before continuing.        ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

SEP = "=" * 55
print(SEP)
print("VALIDATION REPORT")
print(SEP)

errors, warnings = [], []

def _check(ok, msg_pass, msg_fail, is_warning=False):
    if ok:
        print(f"  ✅  {msg_pass}")
    else:
        marker = "⚠️ " if is_warning else "❌ "
        print(f"  {marker}  {msg_fail}")
        (warnings if is_warning else errors).append(msg_fail)

# Row count
_check(len(master) >= EXPECTED_MIN_INCIDENTS,
       f"Row count: {len(master):,} (≥ {EXPECTED_MIN_INCIDENTS:,} expected)",
       f"Row count {len(master):,} is below expected minimum {EXPECTED_MIN_INCIDENTS:,}")

# No duplicate Incident IDs
dupes = master["Incident ID"].duplicated().sum()
_check(dupes == 0, "No duplicate Incident IDs", f"{dupes} duplicate Incident IDs found")

# Core columns 100% filled
core_cols = ["Incident ID","date","year","title","description",
             "deployer","developer","harmed","report_count","Data Sources"]
for col in core_cols:
    nulls = master[col].isnull().sum()
    _check(nulls == 0, f"{col}: 100% filled", f"{col} has {nulls} nulls (should be 0)")

# MIT coverage
mit_pct = master["Risk Domain"].notna().mean() * 100
_check(mit_pct >= EXPECTED_MIT_COVERAGE,
       f"MIT Risk Domain: {mit_pct:.1f}% filled",
       f"MIT coverage {mit_pct:.1f}% is below expected {EXPECTED_MIT_COVERAGE}%",
       is_warning=True)

# Year range
yr_min, yr_max = master["year"].min(), master["year"].max()
_check(yr_min >= 1980 and yr_max >= 2024,
       f"Year range: {yr_min} → {yr_max}",
       f"Unexpected year range: {yr_min} → {yr_max}",
       is_warning=True)

# No row explosion from joins
_check(len(master) == len(inc_core),
       f"No row explosion from joins ({len(master):,} rows)",
       f"Row explosion: master has {len(master):,} but incidents has {len(inc_core):,}")

# Lives lost sanity check
max_lives = master["Lives Lost"].max() if "Lives Lost" in master.columns else 0
print(f"  ℹ️   Max Lives Lost: {max_lives}")

print(f"\n{SEP}")
if errors:
    print(f"  ❌  {len(errors)} ERROR(S) — do not export until resolved:")
    for e in errors: print(f"     • {e}")
else:
    print("  ✅  All checks passed — safe to export")
if warnings:
    print(f"  ⚠️   {len(warnings)} WARNING(S) — review but export is still safe:")
    for w in warnings: print(f"     • {w}")
print(SEP)

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  💾  CELL 12 — EXPORT  (no editing needed)                                  ║
# ║  Writes a 3-sheet Excel file and auto-downloads it in Google Colab.         ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

# ── Style constants ────────────────────────────────────────────────────────────
HDR_FILLS = {
    "identity" : PatternFill("solid", fgColor="1F3864"),
    "coverage" : PatternFill("solid", fgColor="2E4057"),
    "mit"      : PatternFill("solid", fgColor="1A6B3C"),
    "gmf"      : PatternFill("solid", fgColor="7B3F00"),
    "cset"     : PatternFill("solid", fgColor="4A235A"),
}
GROUP_HEX  = {k: v.fgColor.rgb for k, v in HDR_FILLS.items()}
HDR_FONT   = Font(name="Arial", bold=True, color="FFFFFF", size=9)
BODY_FONT  = Font(name="Arial", size=9)
ALT_FILL   = PatternFill("solid", fgColor="F5F5F5")
WHITE_FILL = PatternFill("solid", fgColor="FFFFFF")
_s         = Side(style="thin", color="D0D0D0")
BORDER     = Border(left=_s, right=_s, top=_s, bottom=_s)

# Column → colour group mapping (auto-derived from config)
_identity_cols = ["Incident ID","date","year","title","description","deployer","developer","harmed"]
_coverage_cols = ["Data Sources","report_count"]
_mit_cols      = ["Risk Domain","Risk Subdomain","Responsible Entity","Intent","Timing"]
_gmf_cols      = ["AI Goal","AI Technology","Technical Failure"]

def _col_group(c):
    if c in _identity_cols: return "identity"
    if c in _coverage_cols: return "coverage"
    if c in _mit_cols:      return "mit"
    if c in _gmf_cols:      return "gmf"
    return "cset"

GROUP_LABELS = {
    "identity" : "● INCIDENT IDENTITY  (incidents.csv — 100% complete)",
    "coverage" : "● COVERAGE  (derived)",
    "mit"      : "● RISK CLASSIFICATION — MIT  (~91% coverage)",
    "gmf"      : "● TECHNICAL ANALYSIS — GMF  (~24% coverage)",
    "cset"     : "● POLICY DETAIL — CSETv1  (~16% coverage)",
}


def _write_master_sheet(wb, df):
    ws   = wb.create_sheet("Master Dataset")
    cols = df.columns.tolist()
    n    = len(cols)

    # Row 1: title banner
    ws.merge_cells(start_row=1, start_column=1, end_row=1, end_column=n)
    c = ws.cell(row=1, column=1,
                value=f"AI Incident Database — Master Dataset  |  {len(df):,} incidents  |  {n} columns")
    c.font = Font(name="Arial", bold=True, size=11, color="FFFFFF")
    c.fill = PatternFill("solid", fgColor="0D1B2A")
    c.alignment = Alignment(horizontal="left", vertical="center")
    ws.row_dimensions[1].height = 24

    # Row 2: colour-coded group banners
    curr_grp, grp_start = None, 1
    for ci, col in enumerate(cols, 1):
        grp = _col_group(col)
        if grp != curr_grp:
            if curr_grp is not None:
                ws.merge_cells(start_row=2, start_column=grp_start, end_row=2, end_column=ci-1)
                cell = ws.cell(row=2, column=grp_start)
                cell.value     = GROUP_LABELS.get(curr_grp, "")
                cell.font      = Font(name="Arial", bold=True, size=8, color="FFFFFF")
                cell.fill      = PatternFill("solid", fgColor=GROUP_HEX[curr_grp])
                cell.alignment = Alignment(horizontal="left", vertical="center")
            curr_grp, grp_start = grp, ci
    ws.merge_cells(start_row=2, start_column=grp_start, end_row=2, end_column=n)
    cell = ws.cell(row=2, column=grp_start)
    cell.value     = GROUP_LABELS.get(curr_grp, "")
    cell.font      = Font(name="Arial", bold=True, size=8, color="FFFFFF")
    cell.fill      = PatternFill("solid", fgColor=GROUP_HEX[curr_grp])
    cell.alignment = Alignment(horizontal="left", vertical="center")
    ws.row_dimensions[2].height = 18

    # Row 3: column headers
    for ci, col in enumerate(cols, 1):
        cell = ws.cell(row=3, column=ci, value=col)
        cell.font      = HDR_FONT
        cell.fill      = HDR_FILLS[_col_group(col)]
        cell.alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)
        cell.border    = BORDER
    ws.row_dimensions[3].height = 36

    # Data rows
    for ri, row in enumerate(df.itertuples(index=False), start=4):
        fill = ALT_FILL if ri % 2 == 0 else WHITE_FILL
        for ci, val in enumerate(row, start=1):
            if pd.isna(val):                        val = None
            elif hasattr(val, "strftime"):          val = val.strftime("%Y-%m-%d")
            elif isinstance(val, float) and val == int(val): val = int(val)
            cell = ws.cell(row=ri, column=ci, value=val)
            cell.font      = BODY_FONT
            cell.fill      = fill
            cell.border    = BORDER
            cell.alignment = Alignment(vertical="top", wrap_text=False)

    # Column widths
    widths = {
        "Incident ID":11,"date":13,"year":7,"title":40,"description":50,
        "deployer":25,"developer":25,"harmed":25,"Data Sources":22,"report_count":10,
        "Risk Domain":30,"Risk Subdomain":42,"Responsible Entity":16,"Intent":14,"Timing":16,
        "AI Goal":35,"AI Technology":30,"Technical Failure":35,
    }
    for ci, col in enumerate(cols, 1):
        ws.column_dimensions[get_column_letter(ci)].width = widths.get(col, 18)

    ws.freeze_panes = ws.cell(row=4, column=3)
    ws.auto_filter.ref = f"A3:{get_column_letter(n)}3"


def _write_dict_sheet(wb, df):
    ws = wb.create_sheet("Data Dictionary")
    ws.merge_cells("A1:E1")
    c = ws.cell(row=1, column=1, value="Data Dictionary — AI Incident Database Master Dataset")
    c.font = Font(name="Arial", bold=True, size=12, color="FFFFFF")
    c.fill = PatternFill("solid", fgColor="0D1B2A")
    c.alignment = Alignment(horizontal="left", vertical="center")
    ws.row_dimensions[1].height = 26

    for ci, h in enumerate(["Column","Group","Source","Fill Rate","Description"], 1):
        cell = ws.cell(row=2, column=ci, value=h)
        cell.font      = HDR_FONT
        cell.fill      = PatternFill("solid", fgColor="1F3864")
        cell.alignment = Alignment(horizontal="center", vertical="center")
        cell.border    = BORDER

    # Auto-generate entries from actual master columns
    descriptions = {
        "Incident ID"       : "Primary join key. Unique per incident.",
        "date"              : "Date harm occurred (editor-resolved).",
        "year"              : "Year derived from date — covers all incidents. Best for time-series.",
        "title"             : "Short editor-written title of the incident.",
        "description"       : "1–3 sentence neutral summary of what happened.",
        "deployer"          : "Who deployed the AI. Cleaned from JSON slug format.",
        "developer"         : "Who built the AI system.",
        "harmed"            : "Who was harmed or nearly harmed.",
        "Data Sources"      : "Which taxonomies classified this incident: MIT | GMF | CSETv1.",
        "report_count"      : "Number of linked news articles. Proxy for media attention.",
        "Risk Domain"       : "High-level risk category (7 values). Best for broad trend analysis.",
        "Risk Subdomain"    : "Granular sub-category (21 values) nested under Risk Domain.",
        "Responsible Entity": "Who caused the risk: AI / Human / Other.",
        "Intent"            : "Intentional vs Unintentional vs Other.",
        "Timing"            : "Pre-deployment vs Post-deployment.",
        "AI Goal"           : "What the AI was trying to do (e.g. Autonomous Driving).",
        "AI Technology"     : "ML/AI technique used (e.g. Object Detection).",
        "Technical Failure" : "What technically failed (e.g. Generalization Failure).",
        "Harm Domain"       : "Whether harm occurred in a recognised domain: yes / no / maybe.",
        "Tangible Harm"     : "Level of tangible harm: definitively occurred / near miss / none.",
        "AI Harm Level"     : "AI's contribution to harm severity.",
        "Rights Violation"  : "Whether a legal or human rights violation occurred.",
        "Lives Lost"        : "Fatality count.",
        "Injuries"          : "Injury count.",
        "Sector of Deployment": "Industry sector (ISIC classification).",
        "Location Region"   : "World region: North America / Europe / Asia / Global.",
        "Country Code"      : "ISO 2-letter country code.",
        "Intentional Harm"  : "Whether harm was intentionally designed into the system.",
        "Autonomy Level"    : "Autonomy1 (human-in-loop) through Autonomy4 (fully autonomous).",
    }

    grp_colors = {"Identity":"1F3864","Coverage":"2E4057","MIT":"1A6B3C","GMF":"7B3F00","CSETv1":"4A235A"}
    src_map    = {"identity":"incidents.csv","coverage":"Derived",
                  "mit":"MIT","gmf":"GMF","cset":"CSETv1"}

    for ri, col in enumerate(df.columns, start=3):
        grp  = _col_group(col)
        grp_label = grp.upper() if grp not in ("identity","coverage") else grp.capitalize()
        fill_pct = f"{df[col].notna().mean()*100:.0f}%"
        desc     = descriptions.get(col, "—")
        bg       = ALT_FILL if ri % 2 == 0 else WHITE_FILL
        for ci, val in enumerate([col, grp_label, src_map.get(grp,"—"), fill_pct, desc], 1):
            cell = ws.cell(row=ri, column=ci, value=val)
            cell.fill      = bg
            cell.border    = BORDER
            cell.alignment = Alignment(vertical="top", wrap_text=True)
            cell.font = (Font(name="Arial", size=9, bold=True, color=grp_colors.get(grp_label, "000000"))
                         if ci == 2 else BODY_FONT)

    for col_letter, w in zip(["A","B","C","D","E"], [28,12,14,10,72]):
        ws.column_dimensions[col_letter].width = w
    ws.freeze_panes = "A3"


def _write_coverage_sheet(wb, df):
    ws = wb.create_sheet("Coverage Map")
    ws.merge_cells("A1:D1")
    c = ws.cell(row=1, column=1, value="Coverage Map — What you can analyse at each taxonomy level")
    c.font = Font(name="Arial", bold=True, size=11, color="FFFFFF")
    c.fill = PatternFill("solid", fgColor="0D1B2A")
    c.alignment = Alignment(horizontal="left", vertical="center")
    ws.row_dimensions[1].height = 24

    for ci, h in enumerate(["Data Sources","Incidents","% of Total","What you can analyse"], 1):
        cell = ws.cell(row=2, column=ci, value=h)
        cell.font = HDR_FONT
        cell.fill = PatternFill("solid", fgColor="1F3864")
        cell.alignment = Alignment(horizontal="center", vertical="center")
        cell.border = BORDER

    analysis_map = {
        "MIT"                 : "Risk domain trends, intent, timing, entity — the broadest lens",
        "MIT | GMF"           : "All MIT analysis + technical failures, AI goals, technology types",
        "MIT | GMF | CSETv1"  : "Full picture: risk + technical + policy + sector + geography + harm",
        "MIT | CSETv1"        : "Risk + policy: sector, lives lost, location, harm level, rights",
        "None"                : "Title, description, deployer, developer, harmed, report count only",
    }

    total     = len(df)
    breakdown = df["Data Sources"].value_counts().reset_index()
    breakdown.columns = ["Data Sources", "Count"]

    for ri, row in enumerate(breakdown.itertuples(index=False), start=3):
        bg  = ALT_FILL if ri % 2 == 0 else WHITE_FILL
        pct = f"{row.Count/total*100:.1f}%"
        for ci, val in enumerate([row._0, row.Count, pct, analysis_map.get(row._0, "—")], 1):
            cell = ws.cell(row=ri, column=ci, value=val)
            cell.font = BODY_FONT; cell.fill = bg
            cell.border = BORDER
            cell.alignment = Alignment(vertical="top", wrap_text=True)

    for ci, val in enumerate(["TOTAL", total, "100%", ""], 1):
        cell = ws.cell(row=len(breakdown)+3, column=ci, value=val)
        cell.font   = Font(name="Arial", bold=True, size=9)
        cell.fill   = PatternFill("solid", fgColor="D9E2F3")
        cell.border = BORDER

    for col_letter, w in zip(["A","B","C","D"], [24,14,12,68]):
        ws.column_dimensions[col_letter].width = w


# Write and save
wb = Workbook()
wb.remove(wb.active)
_write_master_sheet(wb, master)
_write_dict_sheet(wb, master)
_write_coverage_sheet(wb, master)
wb.save(OUTPUT_PATH)

print(f"✅  Saved: {OUTPUT_PATH}")
print(f"   Sheets : {[s.title for s in wb.worksheets]}")
print(f"   Master : {len(master):,} rows × {len(master.columns)} columns")

# Auto-download in Google Colab
if IN_COLAB:
    print("\n📥  Starting download to your computer...")
    colab_files.download(str(OUTPUT_PATH))

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  📊  CELL 13 — SUMMARY STATISTICS  (no editing needed)                      ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

SEP = "=" * 55
print(SEP)
print("MASTER DATASET — FINAL SUMMARY")
print(SEP)

print(f"\n  Total incidents  : {len(master):>6,}")
print(f"  Total columns    : {len(master.columns):>6}")
print(f"  Date range       : {int(master['year'].min())} → {int(master['year'].max())}")
print(f"  Output file      : {OUTPUT_PATH}")

print(f"\n  Coverage by taxonomy:")
for src, cnt in master["Data Sources"].value_counts().items():
    bar = "█" * int(cnt / 30)
    print(f"    {src:<28}  {cnt:>5,}  ({cnt/len(master)*100:.1f}%)  {bar}")

print(f"\n  MIT Risk Domains:")
for domain, cnt in master["Risk Domain"].value_counts().items():
    bar = "█" * int(cnt / 30)
    print(f"    {bar:<15} {cnt:>4}  {domain}")

print(f"\n  Incidents by year (2015 onwards):")
year_counts = master[master["year"] >= 2015]["year"].value_counts().sort_index()
for yr, cnt in year_counts.items():
    bar = "█" * int(cnt / 10)
    print(f"    {int(yr)}  {bar:<40} {cnt}")

print(f"\n  Top 10 deployers:")
all_deployers = []
for v in master["deployer"].dropna():
    all_deployers.extend(d.strip().lower() for d in str(v).split(","))
for entity, cnt in collections.Counter(all_deployers).most_common(10):
    print(f"    {cnt:>4}  {entity}")

print(f"\n{SEP}")
print(f"  ✅  Master dataset ready for analysis")
print(SEP)